In [2]:
import keras
from keras import models
from keras import layers
keras.__version__

'2.3.1'

In [3]:
# import and "clean" data

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical ## we need that as our labels cannot be represented by a single output node

base_dir = '/Course notebooks/paintings'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

total_no_of_training_images = 210  ## Keras seems to lack the ability to count, so we set this manually:
total_no_of_test_images = 30
total_no_of_validation_images = 60

In [ ]:
# Define model

In [4]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [5]:
# train model

In [6]:
# validation